2017.4.15 晚
## 准确率与训练集大小
一般情况下，训练越大，准确率越高，在某一一个值附近趋于平缓，不再增加。但实际中，有时候获取数据的代价非常昂贵，甚至只能获取一小部分

In [48]:
import pickle

enron_data = pickle.load(open("E:\\machine-learning\\Data_Mining_for_Tinghua\\Machine_Learning_Udacity\\ud120-projects\\final_project\\final_project_dataset.pkl", "r"))

# 查看人数
print len(enron_data)
# 查看整个数据集
#print enron_data

# 查看每个人具有的特征数
print len(enron_data["SKILLING JEFFREY K"])


# 安然数据中POI的数目
count = 0
person_names = enron_data.keys()
for person_name in person_names:
    if enron_data[person_name]["poi"]==1:
        count += 1
print 'the number of POi',count

# 兑入poi_names 并统计poi_names 条数 竟然有一行网址和空格！所以是35
file_path = 'E:\\machine-learning\\Data_Mining_for_Tinghua\\Machine_Learning_Udacity\\ud120-projects\\final_project\\poi_names.txt'
with open (file_path,'r') as f1:
    context = f1.readlines()
print 'the number of poi_names',len(context)

# James Prentice下的股票

# person_names = sorted(enron_data.keys())
# for person_name in person_names:
#     print person_name
    
enron_data['PRENTICE JAMES']
enron_data['PRENTICE JAMES']['total_stock_value']

# # 来自 Wesley Colwell 的发给嫌疑人的电子邮件
enron_data['COLWELL WESLEY']   

# Jeffrey K Skilling 行使的股票期权价值是多少？
enron_data['SKILLING JEFFREY K']

# 这三个人（Lay、Skilling 和 Fastow）当中，谁拿回家的钱最多（“total_payments”特征的最大值）？
print enron_data['SKILLING JEFFREY K']['total_payments']
print enron_data['LAY KENNETH L']['total_payments']
print enron_data['FASTOW ANDREW S']['total_payments']

# 此数据集中有多少雇员有量化的工资？已知的邮箱地址是否可用？

count_salary = 0
count_email = 0
person_names = enron_data.keys()
for person_name in person_names:
    if enron_data[person_name]["salary"]!='NaN':
        count_salary += 1
    if enron_data[person_name]["email_address"]!='NaN':
        count_email += 1
print count_salary
print count_email




146
21
the number of POi 18
the number of poi_names 37
8682716
103559793
2424083
95
111


In [ ]:
# 我们编写了一些辅助函数（tools/feature_format.py 中的 featureFormat() 和 targetFeatureSplit()），
#它们可以获取特征名的列表和数据字典，然后返回 numpy 数组。

#如果特征没有某个特定人员的值，此函数还会用 0（零）替换特征值。
#!/usr/bin/python

""" 
    A general tool for converting data from the
    dictionary format to an (n x k) python list that's 
    ready for training an sklearn algorithm

    n--no. of key-value pairs in dictonary
    k--no. of features being extracted

    dictionary keys are names of persons in dataset
    dictionary values are dictionaries, where each
        key-value pair in the dict is the name
        of a feature, and its value for that person

    In addition to converting a dictionary to a numpy 
    array, you may want to separate the labels from the
    features--this is what targetFeatureSplit is for

    so, if you want to have the poi label as the target,
    and the features you want to use are the person's
    salary and bonus, here's what you would do:

    feature_list = ["poi", "salary", "bonus"] 
    data_array = featureFormat( data_dictionary, feature_list )
    label, features = targetFeatureSplit(data_array)

    the line above (targetFeatureSplit) assumes that the
    label is the _first_ item in feature_list--very important
    that poi is listed first!
"""


import numpy as np

def featureFormat( dictionary, features, remove_NaN=True, remove_all_zeroes=True, remove_any_zeroes=False, sort_keys = False):
    """ convert dictionary to numpy array of features
        remove_NaN = True will convert "NaN" string to 0.0
        remove_all_zeroes = True will omit any data points for which
            all the features you seek are 0.0
        remove_any_zeroes = True will omit any data points for which
            any of the features you seek are 0.0
        sort_keys = True sorts keys by alphabetical order. Setting the value as
            a string opens the corresponding pickle file with a preset key
            order (this is used for Python 3 compatibility, and sort_keys
            should be left as False for the course mini-projects).
        NOTE: first feature is assumed to be 'poi' and is not checked for
            removal for zero or missing values.
    """


    return_list = []

    # Key order - first branch is for Python 3 compatibility on mini-projects,
    # second branch is for compatibility on final project.
    if isinstance(sort_keys, str):
        import pickle
        keys = pickle.load(open(sort_keys, "rb"))
    elif sort_keys:
        keys = sorted(dictionary.keys())
    else:
        keys = dictionary.keys()

    for key in keys:
        tmp_list = []
        for feature in features:
            try:
                dictionary[key][feature]
            except KeyError:
                print "error: key ", feature, " not present"
                return
            value = dictionary[key][feature]
            if value=="NaN" and remove_NaN:
                value = 0
            tmp_list.append( float(value) )

        # Logic for deciding whether or not to add the data point.
        append = True
        # exclude 'poi' class as criteria.
        if features[0] == 'poi':
            test_list = tmp_list[1:]
        else:
            test_list = tmp_list
        ### if all features are zero and you want to remove
        ### data points that are all zero, do that here
        if remove_all_zeroes:
            append = False
            for item in test_list:
                if item != 0 and item != "NaN":
                    append = True
                    break
        ### if any features for a given data point are zero
        ### and you want to remove data points with any zeroes,
        ### handle that here
        if remove_any_zeroes:
            if 0 in test_list or "NaN" in test_list:
                append = False
        ### Append the data point if flagged for addition.
        if append:
            return_list.append( np.array(tmp_list) )

    return np.array(return_list)


def targetFeatureSplit( data ):
    """ 
        given a numpy array like the one returned from
        featureFormat, separate out the first feature
        and put it into its own list (this should be the 
        quantity you want to predict)

        return targets and features as separate lists

        (sklearn can generally handle both lists and numpy arrays as 
        input formats when training/predicting)
    """

    target = []
    features = []
    for item in data:
        target.append( item[0] )
        features.append( item[1:] )

    return target, features






In [49]:
# （当前的）E+F 数据集中有多少人的薪酬总额被设置了“NaN”？数据集中这些人的比例占多少？
count_nan = 0
person_names = enron_data.keys()
for person_name in person_names:
    if enron_data[person_name]["total_payments"]=='NaN':
        count_nan += 1
print count_nan
print (count_nan/146.0)

# E+F 数据集中有多少 POI 的薪酬总额被设置了“NaN”？这些 POI 占多少比例？

count_na = 0
count = 0
poi_members = []
person_names = enron_data.keys()
for person_name in person_names:
    if enron_data[person_name]["poi"]==1:
        poi_members.append(person_name)
        count += 1
for poi_member in poi_members:
    if enron_data[poi_member]['total_payments'] == 'NaN':
        count_na += 1
print count
print count_na
            
print count_na/float(count) # 当时还不相信自己的结果


# 如果你再次添加了全是 POI 的 10 个数据点，并且对这些雇员的薪酬总额设置了“NaN”，你刚才计算的数字会发生变化。
# 数据集中这些人的数量变成了多少？薪酬总额被设置了“NaN”的雇员数变成了多少？


# 数据集中的 POI 数量变成了多少？股票总值被设置了“NaN”的 POI 占多少比例？
count_stock = 0
for poi_member in poi_members:
    if enron_data[poi_member]['total_stock_value'] == 'NaN':
        count_stock += 1
print 'the number of count_stock',count_stock




21
0.143835616438
18
0
0.0
the number of count_stock 0


## 读写文件巩固
读写文件就是请求操作系统打开一个文件对象（通常称为文件描述符），然后，通过操作系统提供的接口从这个文件对象中读取数据（读文件），或者把数据写入这个文件对象（写文件）。

### 读文件 
读文件是将磁盘上的文件读到内存中；默认都是读取文本文件，并且是UTF-8编码的文本文件。要读取二进制文件，比如图片、视频等等，用'rb'模式打开文件即可
+ **read** read()方法可以一次读取文件的全部内容，Python把内容读到内存，用一个str对象表

```python
# example for read
with open('/path/to/file', 'r') as f:
    print(f.read())
```
+ **read(size)** 如果文件内容过大，可以反复调用read(size)方法，每次最多读取size个字节的内容
+ **readline** 调用readline()可以每次读取一行内容，
+ **readlines** 调用readlines()一次读取所有内容并按行返回list
+ **读非utf-8编码** 需要给open()函数传入encoding参数，例如，读取GBK编码的文件
`f = open('/Users/michael/gbk.txt', 'r', encoding='gbk')`
+ **UnicodeDecodeError** 遇到到有些编码不规范的文件，你可能会遇到UnicodeDecodeError，因为在文本文件中可能夹杂了一些非法编码的字符。遇到这种情况，open()函数还接收一个errors参数，表示如果遇到编码错误后如何处理。最简单的方式是直接忽略：
`f = open('/Users/michael/gbk.txt', 'r', encoding='gbk', errors='ignore')
`
### 写文件
通过操作系统将文件写到磁盘中，操作系统往往不会立刻把数据写入磁盘，而是放到内存缓存起来，空闲的时候再慢慢写入。只有调用close()方法时，操作系统才保证把没有写入的数据全部写入磁盘。忘记调用close()的后果是数据可能只写了一部分到磁盘，剩下的丢失了。所以，还是用with语句来得保险.要写入特定编码的文本文件，请给open()函数传入encoding参数，将字符串自动转换成指定编码。

```python
with open('/Users/michael/test.txt', 'w') as f:
    f.write('Hello, world!')
```
